Edge index excluded from dataloader (used it as a global variable)

In [14]:
import os ; os.chdir(r"D:\Git_repos\Photonic_GCN")
from constants import edges_table, nodes_groups
# !pip install torch torch-geometric
import torch , pandas as pd
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.utils.data import TensorDataset, random_split , DataLoader as DataLoader_primary
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


Inputs

In [16]:
BATCH_SIZE = 8
LEARNING_RATE = 0.001
LR_SCHEDULER_GAMMA = 0.9
LR_SCHEDULER_STEP_SIZE = 20
EPOCHS = 2000

Edges list declaration

In [17]:
def edges_table_to_list(edges_table):
    edges = []
    for key in edges_table.keys():
        nodes_list = edges_table[key]
        for node in nodes_list:
            if ((key, node) not in edges) and ((node, key) not in edges):
                edges.append((key, node))
    return edges

In [18]:
edge_list = edges_table_to_list(edges_table)  # 107 edges
print(len(edge_list)) 
print(edge_list)

107
[('N69', 'N70'), ('N70', 'N71'), ('N71', 'N72'), ('N72', 'N73'), ('N73', 'N74'), ('N73', 'N60'), ('N74', 'N60'), ('N74', 'N94'), ('N60', 'N45'), ('N60', 'N46'), ('N60', 'N47'), ('N60', 'N61'), ('N60', 'N75'), ('N41', 'N42'), ('N42', 'N43'), ('N43', 'N44'), ('N44', 'N45'), ('N45', 'N46'), ('N46', 'N33'), ('N87', 'N86'), ('N86', 'N85'), ('N85', 'N84'), ('N84', 'N83'), ('N83', 'N82'), ('N83', 'N68'), ('N82', 'N68'), ('N82', 'N94'), ('N68', 'N55'), ('N68', 'N54'), ('N68', 'N53'), ('N68', 'N67'), ('N68', 'N81'), ('N59', 'N58'), ('N58', 'N57'), ('N57', 'N56'), ('N56', 'N55'), ('N55', 'N54'), ('N54', 'N34'), ('N33', 'N61'), ('N33', 'N47'), ('N33', 'N48'), ('N34', 'N52'), ('N34', 'N53'), ('N34', 'N67'), ('N94', 'N61'), ('N94', 'N75'), ('N94', 'N76'), ('N95', 'N80'), ('N95', 'N81'), ('N95', 'N82'), ('N95', 'N67'), ('N67', 'N81'), ('N67', 'N53'), ('N67', 'N80'), ('N67', 'N66'), ('N67', 'N52'), ('N53', 'N52'), ('N53', 'N66'), ('N81', 'N80'), ('N81', 'N66'), ('N80', 'N79'), ('N80', 'N66'), ('N

In [19]:
node_names = list(edges_table.keys())  # 51 nodes
print(node_names)

['N69', 'N70', 'N71', 'N72', 'N73', 'N74', 'N60', 'N41', 'N42', 'N43', 'N44', 'N45', 'N46', 'N87', 'N86', 'N85', 'N84', 'N83', 'N82', 'N68', 'N59', 'N58', 'N57', 'N56', 'N55', 'N54', 'N33', 'N34', 'N94', 'N95', 'N67', 'N53', 'N81', 'N80', 'N52', 'N79', 'N51', 'N65', 'N66', 'N78', 'N64', 'N50', 'N61', 'N75', 'N47', 'N62', 'N76', 'N48', 'N63', 'N77', 'N49']


In [20]:
# Convert edge list to indices
edge_index = torch.tensor([[node_names.index(u), node_names.index(v)] for u, v in edge_list], dtype=torch.long).t().contiguous()
edge_index_orignal = edge_index

edge_index.shape

torch.Size([2, 107])

Data preparation

In [21]:
df = pd.read_excel('data.xlsx')

X = df.iloc[:, :18].values
y = df.iloc[:, 18:19].values

scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32).to(device)

train_size = int(0.7 * len(X_tensor))
val_size = int(0.15 * len(X_tensor))
test_size = len(X_tensor) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(TensorDataset(X_tensor, y_tensor), [train_size, val_size, test_size])

train_dataloader = DataLoader_primary(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader_primary(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader_primary(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [22]:
print(X.shape)
df

(8015, 18)


,dx1,dx2,dx3,dx4,dx5,dx6,dy1,dy2,dy3,dy4,...,dy6,dr1,dr2,dr3,dr4,dr5,dr6,Q,t_q_max,lambda_q_max
0,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,918.130324,0.554622,0.000001
1,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.750000e-08,2.500000e-08,1846.114296,0.682024,0.000001
2,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,3.000000e-08,2.500000e-08,660.817503,1.089219,0.000002
3,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.750000e-08,2.500000e-08,1846.114296,0.682024,0.000001
4,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,3.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.750000e-08,2.500000e-08,1846.114296,0.682024,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8010,-4.000000e-08,3.500000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,1204.550299,0.521705,0.000001
8011,-4.000000e-08,4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,1306.557266,0.549646,0.000001
8012,-4.000000e-08,4.500000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,618.706366,0.701339,0.000002
8013,-4.000000e-08,5.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,610.208722,0.826126,0.000002


In [24]:
nodes2group = dict()

for g in nodes_groups:
    for node in nodes_groups[g]:
        nodes2group[node] = g

Data normalization

dx , dy are between 1.55e-7 and -4e-8

dr is between 1.75e-8 and 3.25e-8 

Almost 98% of Qs are between 500 and 50000

we multiply dx, dy & dr by 1e+7 and Q by 1e-4

In [116]:
normalized_df = df.copy()

normalized_df.iloc[:, 0:18] = normalized_df.iloc[:, 0:18] * 1e+7
normalized_df.iloc[:, 18] = normalized_df.iloc[:, 18] * 1e-4
normalized_df

,dx1,dx2,dx3,dx4,dx5,dx6,dy1,dy2,dy3,dy4,...,dy6,dr1,dr2,dr3,dr4,dr5,dr6,Q,t_q_max,lambda_q_max
0,-0.4,-0.40,-0.4,-0.4,-0.4,-0.45,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.250,0.25,0.091813,0.554622,0.000001
1,-0.4,-0.40,-0.4,-0.4,-0.4,-0.45,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.275,0.25,0.184611,0.682024,0.000001
2,-0.4,-0.40,-0.4,-0.4,-0.4,-0.45,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.300,0.25,0.066082,1.089219,0.000002
3,-0.4,-0.40,-0.4,-0.4,-0.4,-0.45,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.275,0.25,0.184611,0.682024,0.000001
4,-0.4,-0.40,-0.4,-0.4,-0.4,-0.45,0.35,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.275,0.25,0.184611,0.682024,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8010,-0.4,0.35,-0.4,-0.4,-0.4,-0.40,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.250,0.25,0.120455,0.521705,0.000001
8011,-0.4,0.40,-0.4,-0.4,-0.4,-0.40,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.250,0.25,0.130656,0.549646,0.000001
8012,-0.4,0.45,-0.4,-0.4,-0.4,-0.40,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.250,0.25,0.061871,0.701339,0.000002
8013,-0.4,0.50,-0.4,-0.4,-0.4,-0.40,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.250,0.25,0.061021,0.826126,0.000002


Shuffle data

In [117]:
np.random.seed(42)
shuffled_df = normalized_df.sample(frac=1)
shuffled_df

,dx1,dx2,dx3,dx4,dx5,dx6,dy1,dy2,dy3,dy4,...,dy6,dr1,dr2,dr3,dr4,dr5,dr6,Q,t_q_max,lambda_q_max
6438,-0.4,0.40,-0.4,-0.40,-0.40,-0.40,0.40,0.4,0.40,0.35,...,0.40,0.25,0.250,0.25,0.250,0.250,0.225,0.139398,0.515356,0.000001
318,-0.4,-0.25,-0.4,-0.40,-0.45,-0.40,0.40,0.4,0.45,0.40,...,0.10,0.25,0.275,0.25,0.225,0.275,0.225,0.123603,0.691624,0.000002
1882,-0.4,-0.25,-0.4,-0.40,-0.35,-0.40,0.35,0.4,0.40,0.40,...,0.40,0.25,0.250,0.25,0.250,0.250,0.225,0.068667,1.086635,0.000001
5166,-0.4,0.15,-0.4,-0.40,-0.40,-0.40,0.40,0.4,0.35,0.40,...,0.40,0.25,0.250,0.25,0.250,0.250,0.250,0.067726,0.710172,0.000002
4962,-0.4,0.35,-0.4,-0.40,-0.40,-0.40,0.40,0.4,0.40,0.40,...,0.40,0.25,0.250,0.25,0.250,0.250,0.250,0.153093,0.573313,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5226,-0.4,1.20,-0.4,-0.40,-0.40,-0.40,0.40,0.4,0.40,0.40,...,0.40,0.25,0.250,0.25,0.250,0.250,0.250,0.047967,1.072468,0.000002
5390,-0.4,1.20,-0.4,-0.40,-0.40,-0.40,0.40,0.4,0.40,0.40,...,0.40,0.25,0.275,0.25,0.250,0.250,0.250,0.117560,0.513500,0.000001
860,-0.4,-0.40,-0.4,-0.45,-0.40,-0.40,0.40,0.4,0.40,0.35,...,0.15,0.25,0.250,0.25,0.225,0.250,0.250,0.054203,0.900257,0.000001
7603,-0.4,0.90,-0.4,-0.40,-0.40,-0.40,0.45,0.4,0.40,0.40,...,0.40,0.25,0.250,0.25,0.250,0.250,0.250,0.123656,0.697853,0.000001


In [120]:
train_indice_split = 7600

train_df = shuffled_df[:train_indice_split].reset_index()
test_df = shuffled_df[train_indice_split:].reset_index()

In [121]:
train_df

,index,dx1,dx2,dx3,dx4,dx5,dx6,dy1,dy2,dy3,...,dy6,dr1,dr2,dr3,dr4,dr5,dr6,Q,t_q_max,lambda_q_max
0,6438,-0.4,0.40,-0.40,-0.40,-0.40,-0.4,0.40,0.4,0.40,...,0.4,0.25,0.250,0.250,0.250,0.250,0.225,0.139398,0.515356,0.000001
1,318,-0.4,-0.25,-0.40,-0.40,-0.45,-0.4,0.40,0.4,0.45,...,0.1,0.25,0.275,0.250,0.225,0.275,0.225,0.123603,0.691624,0.000002
2,1882,-0.4,-0.25,-0.40,-0.40,-0.35,-0.4,0.35,0.4,0.40,...,0.4,0.25,0.250,0.250,0.250,0.250,0.225,0.068667,1.086635,0.000001
3,5166,-0.4,0.15,-0.40,-0.40,-0.40,-0.4,0.40,0.4,0.35,...,0.4,0.25,0.250,0.250,0.250,0.250,0.250,0.067726,0.710172,0.000002
4,4962,-0.4,0.35,-0.40,-0.40,-0.40,-0.4,0.40,0.4,0.40,...,0.4,0.25,0.250,0.250,0.250,0.250,0.250,0.153093,0.573313,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,4735,-0.4,1.15,-0.45,-0.40,-0.40,-0.4,0.40,0.4,0.40,...,0.4,0.25,0.250,0.250,0.250,0.250,0.250,0.048688,0.866636,0.000002
7596,7713,-0.4,0.15,-0.40,-0.40,-0.40,-0.4,0.40,0.4,0.40,...,0.4,0.25,0.250,0.250,0.250,0.225,0.250,0.134603,0.606360,0.000001
7597,1218,-0.4,1.45,-0.40,-0.35,-0.35,-0.4,0.35,0.4,0.40,...,0.3,0.25,0.225,0.225,0.275,0.250,0.250,0.091626,0.551384,0.000001
7598,5254,-0.4,0.55,-0.40,-0.40,-0.35,-0.4,0.40,0.4,0.40,...,0.4,0.25,0.250,0.250,0.250,0.250,0.250,0.056174,0.851991,0.000002


In [122]:
test_df

,index,dx1,dx2,dx3,dx4,dx5,dx6,dy1,dy2,dy3,...,dy6,dr1,dr2,dr3,dr4,dr5,dr6,Q,t_q_max,lambda_q_max
0,7618,-0.4,1.45,-0.4,-0.40,-0.4,-0.45,0.45,0.35,0.40,...,0.40,0.25,0.250,0.25,0.250,0.25,0.25,0.047586,0.861553,0.000002
1,1527,-0.4,-0.40,-0.4,-0.40,-0.4,-0.40,0.40,0.40,0.40,...,0.40,0.25,0.250,0.25,0.250,0.25,0.25,0.075180,0.602837,0.000001
2,3987,-0.4,0.75,-0.4,-0.40,-0.4,-0.40,0.40,0.40,0.45,...,0.40,0.25,0.250,0.25,0.250,0.25,0.25,0.137036,0.527903,0.000001
3,7906,-0.4,1.15,-0.4,-0.40,-0.4,-0.40,0.40,0.35,0.40,...,0.40,0.25,0.250,0.25,0.250,0.25,0.25,0.095046,0.557288,0.000001
4,1300,-0.4,-0.40,-0.4,-0.40,-0.4,-0.40,0.40,0.40,0.40,...,0.30,0.25,0.250,0.25,0.250,0.25,0.25,0.131589,0.525373,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,5226,-0.4,1.20,-0.4,-0.40,-0.4,-0.40,0.40,0.40,0.40,...,0.40,0.25,0.250,0.25,0.250,0.25,0.25,0.047967,1.072468,0.000002
411,5390,-0.4,1.20,-0.4,-0.40,-0.4,-0.40,0.40,0.40,0.40,...,0.40,0.25,0.275,0.25,0.250,0.25,0.25,0.117560,0.513500,0.000001
412,860,-0.4,-0.40,-0.4,-0.45,-0.4,-0.40,0.40,0.40,0.40,...,0.15,0.25,0.250,0.25,0.225,0.25,0.25,0.054203,0.900257,0.000001
413,7603,-0.4,0.90,-0.4,-0.40,-0.4,-0.40,0.45,0.40,0.40,...,0.40,0.25,0.250,0.25,0.250,0.25,0.25,0.123656,0.697853,0.000001


In [123]:
train_x_list = []
train_y_list = []

for n in range(len(train_df)):
    x = np.zeros((51, 3))
    y = np.array(train_df["Q"][n])
    for idx, node in enumerate(node_names):
        group = nodes2group[node][1:]
        node_features = np.array((train_df["dx" + group][n], train_df["dy" + group][n], train_df["dr" + group][n],))
        x[idx] = node_features
    train_x_list.append(x)
    train_y_list.append(y)

train_x = torch.from_numpy(np.stack(train_x_list, axis=0))
train_y = torch.from_numpy(np.stack(train_y_list, axis=0))
print(train_x.shape)
print(train_y.shape)
print(edge_index.shape)

torch.Size([7600, 51, 3])
torch.Size([7600])
torch.Size([2, 107])


In [124]:
test_x_list = []
test_y_list = []

for n in range(len(test_df)):
    x = np.zeros((51, 3))
    y = np.array(test_df["Q"][n])
    for idx, node in enumerate(node_names):
        group = nodes2group[node][1:]
        node_features = np.array((test_df["dx" + group][n], test_df["dy" + group][n], test_df["dr" + group][n],))
        x[idx] = node_features
    test_x_list.append(x)
    test_y_list.append(y)

test_x = torch.from_numpy(np.stack(test_x_list, axis=0))
test_y = torch.from_numpy(np.stack(test_y_list, axis=0))
print(test_x.shape)
print(test_y.shape)
print(edge_index.shape)

torch.Size([415, 51, 3])
torch.Size([415])
torch.Size([2, 107])


In [26]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, x, y):
        """
        Args:
            x (Tensor): Tensor of shape (8015, 51, 3) containing the input data.
            labels (Tensor): Tensor of shape (8015,) containing the labels.
        """
        self.x = x.float()  # Ensure x is float32
        self.y = y.float()  # Ensure y is float32
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [129]:
dataset_train = MyDataset(train_x, train_y)
dataloader_train = DataLoader(train_dataset, batch_size=1, shuffle=True)

dataset_test = MyDataset(test_x, test_y)
dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=True)

Model setup

In [35]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)  # Output 1 feature per node

    def forward(self, data, edge_index):
        x = data
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(Net, self).__init__()
        self.gcn = GCN(in_channels, hidden_channels)
        self.fc1 = torch.nn.Linear(51, 128)  # Input size should match number of nodes
        self.fc2 = torch.nn.Linear(128, 1)   # Output a single value

    def forward(self, data, edge_index):
        x = self.gcn(data, edge_index)
        x = x.view(1, -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Model parameters
in_channels = 3
hidden_channels = 16

# Initialize model
model = Net(in_channels, hidden_channels)
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = StepLR(optimizer, step_size=LR_SCHEDULER_STEP_SIZE, gamma=LR_SCHEDULER_GAMMA)

Training

In [36]:
# edge_index = edge_index.to(device) 
best_models_dir = r"D:\Git_repos\Photonic_GCN"

best_loss = 100
best_loss_epoch = 1

model.train()
for epoch in range(1, EPOCHS):
    total_loss = 0
    for batch in dataloader_train:
        optimizer.zero_grad()
        x, y = batch
        x = x.float()
        y = y.float() 
        # x.to(device); y.to(device) 
        out = model(x, edge_index) 
        y = y.view(out.shape)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() 

    scheduler.step()
    avg_loss = total_loss / len(dataloader_train)  # Compute average loss for the epoch
    print(f'Epoch {epoch}, Loss: {avg_loss:.6f} *** Best loss so far: {best_loss:.6f} at epoch {best_loss_epoch}')

    if avg_loss < best_loss:
        best_loss = avg_loss
        best_loss_epoch = epoch
        best_model_path = os.path.join(best_models_dir, "best_model.pt")
        torch.save(model.state_dict(), best_model_path)
        # print(f"Best model saved with loss:  {avg_loss}%")


Epoch 1, Loss: 0.020333 *** Best loss so far: 100.000000 at epoch 1
Epoch 2, Loss: 0.020047 *** Best loss so far: 0.020333 at epoch 1
Epoch 3, Loss: 0.020034 *** Best loss so far: 0.020047 at epoch 2
Epoch 4, Loss: 0.019932 *** Best loss so far: 0.020034 at epoch 3
Epoch 5, Loss: 0.019908 *** Best loss so far: 0.019932 at epoch 4
Epoch 6, Loss: 0.019843 *** Best loss so far: 0.019908 at epoch 5
Epoch 7, Loss: 0.019897 *** Best loss so far: 0.019843 at epoch 6
Epoch 8, Loss: 0.019835 *** Best loss so far: 0.019843 at epoch 6
Epoch 9, Loss: 0.019868 *** Best loss so far: 0.019835 at epoch 8
Epoch 10, Loss: 0.019700 *** Best loss so far: 0.019835 at epoch 8
Epoch 11, Loss: 0.019810 *** Best loss so far: 0.019700 at epoch 10
Epoch 12, Loss: 0.019821 *** Best loss so far: 0.019700 at epoch 10
Epoch 13, Loss: 0.019843 *** Best loss so far: 0.019700 at epoch 10
Epoch 14, Loss: 0.019876 *** Best loss so far: 0.019700 at epoch 10
Epoch 15, Loss: 0.019783 *** Best loss so far: 0.019700 at epoch 

Testing

In [128]:
print(test_x.shape)
print(test_y.shape)

torch.Size([415, 51, 3])
torch.Size([415])


In [139]:
for batch in dataloader_test:
    x, y = batch    
    x = x.float()
    y = y.float() 
    model.eval()
    with torch.no_grad():
        out = model(x, edge_index)
        
        prediction = float(out[0][0] * 1e4)
        real = float(y[0] * 1e4)
        print(f"pred: {prediction:.1f}")
        print(f"real: {real:.1f}")
        print(" -------------------------------- ")

pred: 975.5
real: 825.7
 -------------------------------- 
pred: 712.5
real: 800.0
 -------------------------------- 
pred: 1060.7
real: 1021.5
 -------------------------------- 
pred: 693.9
real: 1363.1
 -------------------------------- 
pred: 1062.4
real: 1185.0
 -------------------------------- 
pred: 832.9
real: 1321.5
 -------------------------------- 
pred: 970.2
real: 1200.9
 -------------------------------- 
pred: 675.5
real: 1651.3
 -------------------------------- 
pred: 881.4
real: 810.6
 -------------------------------- 
pred: 946.0
real: 894.7
 -------------------------------- 
pred: 1461.5
real: 1408.3
 -------------------------------- 
pred: 539.6
real: 512.4
 -------------------------------- 
pred: 946.0
real: 894.7
 -------------------------------- 
pred: 1209.9
real: 1204.6
 -------------------------------- 
pred: 779.0
real: 662.2
 -------------------------------- 
pred: 557.4
real: 483.9
 -------------------------------- 
pred: 838.7
real: 946.3
 -------------------